## Module 3 Assignment 3: House Prices: Advanced Regression Techniques EDA 

### I. EDA -Set up

In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from scipy.stats import shapiro
from scipy import stats
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [81]:
df_train = pd.read_csv("train-2.csv")
train = pd.read_csv("train-2.csv")
x_testset = pd.read_csv("test.csv")

### II. EDA - Data Wrangling

In [82]:
df_train.info() #Need to Drop the columns with more null value row
df_train.shape
df_train.head()
df_trainID = df_train['Id']
x_testID = x_testset['Id']
y = df_train[['SalePrice']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

(1460, 81)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [83]:
df_train = pd.concat((df_train,x_testset),ignore_index=True)
df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2915,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2916,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
2917,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


In [84]:
df2_train = df_train.drop(columns=['Id', 'LotFrontage',
                                   'Alley',
                                   'FireplaceQu',
                                  'PoolQC',
                                  'Fence',
                                  'MiscFeature'])
df2_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   MSZoning       2915 non-null   object 
 2   LotArea        2919 non-null   int64  
 3   Street         2919 non-null   object 
 4   LotShape       2919 non-null   object 
 5   LandContour    2919 non-null   object 
 6   Utilities      2917 non-null   object 
 7   LotConfig      2919 non-null   object 
 8   LandSlope      2919 non-null   object 
 9   Neighborhood   2919 non-null   object 
 10  Condition1     2919 non-null   object 
 11  Condition2     2919 non-null   object 
 12  BldgType       2919 non-null   object 
 13  HouseStyle     2919 non-null   object 
 14  OverallQual    2919 non-null   int64  
 15  OverallCond    2919 non-null   int64  
 16  YearBuilt      2919 non-null   int64  
 17  YearRemodAdd   2919 non-null   int64  
 18  RoofStyl

In [85]:
#Year Remodeling adding minus current year = remodyear
df2_train['remodyear'] = 2022 - df2_train['YearRemodAdd']
df2_train = df2_train.drop(columns = ['YearRemodAdd'])

#Use basement finish area divdes total basement area, find finish ratio
df2_train['FinishRatio'] = df2_train['BsmtUnfSF']/df2_train['TotalBsmtSF']
df2_train = df2_train.drop(columns = ['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF'])

#Use year and month sold to come up with sold year-month 
df2_train['MoSold'] = df2_train.MoSold.map("{:02}".format)
df2_train['SoldTime'] = df2_train['YrSold'].map(str) + df2_train['MoSold'].map(str)
df2_train = df2_train.drop(columns = ['YrSold','MoSold'])


In [86]:
df2_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 71 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   MSZoning       2915 non-null   object 
 2   LotArea        2919 non-null   int64  
 3   Street         2919 non-null   object 
 4   LotShape       2919 non-null   object 
 5   LandContour    2919 non-null   object 
 6   Utilities      2917 non-null   object 
 7   LotConfig      2919 non-null   object 
 8   LandSlope      2919 non-null   object 
 9   Neighborhood   2919 non-null   object 
 10  Condition1     2919 non-null   object 
 11  Condition2     2919 non-null   object 
 12  BldgType       2919 non-null   object 
 13  HouseStyle     2919 non-null   object 
 14  OverallQual    2919 non-null   int64  
 15  OverallCond    2919 non-null   int64  
 16  YearBuilt      2919 non-null   int64  
 17  RoofStyle      2919 non-null   object 
 18  RoofMatl

In [87]:
#separate categorical var to numerical
Cat_col = list(df2_train.dtypes[df2_train.dtypes == object].index)
Cat_col
Num_col = list(df2_train.dtypes[df2_train.dtypes != object ].index)
Num_col

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition',
 'SoldTime']

['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'MasVnrArea',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'SalePrice',
 'remodyear',
 'FinishRatio']

In [88]:
for col in Num_col:
    df2_train[col].fillna(df2_train[col].median(), inplace=True)

df2_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 71 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   MSZoning       2915 non-null   object 
 2   LotArea        2919 non-null   int64  
 3   Street         2919 non-null   object 
 4   LotShape       2919 non-null   object 
 5   LandContour    2919 non-null   object 
 6   Utilities      2917 non-null   object 
 7   LotConfig      2919 non-null   object 
 8   LandSlope      2919 non-null   object 
 9   Neighborhood   2919 non-null   object 
 10  Condition1     2919 non-null   object 
 11  Condition2     2919 non-null   object 
 12  BldgType       2919 non-null   object 
 13  HouseStyle     2919 non-null   object 
 14  OverallQual    2919 non-null   int64  
 15  OverallCond    2919 non-null   int64  
 16  YearBuilt      2919 non-null   int64  
 17  RoofStyle      2919 non-null   object 
 18  RoofMatl

In [89]:
for col in Cat_col:
    df2_train[col].fillna(df2_train[col].mode()[0], inplace=True)

df2_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 71 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2919 non-null   int64  
 1   MSZoning       2919 non-null   object 
 2   LotArea        2919 non-null   int64  
 3   Street         2919 non-null   object 
 4   LotShape       2919 non-null   object 
 5   LandContour    2919 non-null   object 
 6   Utilities      2919 non-null   object 
 7   LotConfig      2919 non-null   object 
 8   LandSlope      2919 non-null   object 
 9   Neighborhood   2919 non-null   object 
 10  Condition1     2919 non-null   object 
 11  Condition2     2919 non-null   object 
 12  BldgType       2919 non-null   object 
 13  HouseStyle     2919 non-null   object 
 14  OverallQual    2919 non-null   int64  
 15  OverallCond    2919 non-null   int64  
 16  YearBuilt      2919 non-null   int64  
 17  RoofStyle      2919 non-null   object 
 18  RoofMatl

In [90]:
#Encoding categorical columns
#one hot encoding nominal var
nominal_var = ['MSZoning', 
               'Street', 
               'LotShape', 
               'LandContour', 
               'Utilities', 
               'LotConfig', 
               'LandSlope', 
               'Neighborhood', 
               'Condition1', 
               'Condition2', 
               'BldgType', 
               'HouseStyle', 
               'RoofStyle', 
               'RoofMatl', 
               'Exterior1st', 
               'Exterior2nd', 
               'MasVnrType', 
               'Foundation', 
               'Heating', 
               'Functional', 
               'GarageType', 
               'SaleType', 
               'SaleCondition']
ordinal_var = ['ExterQual', 
               'ExterCond', 
               'BsmtQual', 
               'BsmtCond', 
               'BsmtExposure', 
               'BsmtFinType1', 
               'BsmtFinType2', 
               'HeatingQC', 
               'CentralAir', 
               'Electrical', 
               'KitchenQual', 
               'GarageFinish', 
               'GarageQual', 
               'GarageCond', 
               'PavedDrive']


In [91]:
one_hot = pd.get_dummies(df2_train[nominal_var])
one_hot

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2915,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2916,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2917,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [92]:
ordinalencoder = OrdinalEncoder()
ordinalencod = ordinalencoder.fit_transform(df2_train[ordinal_var])
ordi_encode = pd.DataFrame(ordinalencod, columns = [ordinal_var])
ordi_encode.head()
ordi_encode.columns = ['ExterQual', 
               'ExterCond', 
               'BsmtQual', 
               'BsmtCond', 
               'BsmtExposure', 
               'BsmtFinType1', 
               'BsmtFinType2', 
               'HeatingQC', 
               'CentralAir', 
               'Electrical', 
               'KitchenQual', 
               'GarageFinish', 
               'GarageQual', 
               'GarageCond', 
               'PavedDrive']#ordinal variable encoded

,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,Electrical,KitchenQual,GarageFinish,GarageQual,GarageCond,PavedDrive
0,2.0,4.0,2.0,3.0,3.0,2.0,5.0,0.0,1.0,4.0,2.0,1.0,4.0,4.0,2.0
1,3.0,4.0,2.0,3.0,1.0,0.0,5.0,0.0,1.0,4.0,3.0,1.0,4.0,4.0,2.0
2,2.0,4.0,2.0,3.0,2.0,2.0,5.0,0.0,1.0,4.0,2.0,1.0,4.0,4.0,2.0
3,3.0,4.0,3.0,1.0,3.0,0.0,5.0,2.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0
4,2.0,4.0,2.0,3.0,0.0,2.0,5.0,0.0,1.0,4.0,2.0,1.0,4.0,4.0,2.0


In [93]:
Num_nnull = df2_train[Num_col]
df3_train = pd.concat([Num_nnull, one_hot , ordi_encode],axis = 'columns')
df3_train

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,MasVnrArea,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,Electrical,KitchenQual,GarageFinish,GarageQual,GarageCond,PavedDrive
0,60,8450,7,5,2003,196.0,856.0,856,854,0,...,2.0,5.0,0.0,1.0,4.0,2.0,1.0,4.0,4.0,2.0
1,20,9600,6,8,1976,0.0,1262.0,1262,0,0,...,0.0,5.0,0.0,1.0,4.0,3.0,1.0,4.0,4.0,2.0
2,60,11250,7,5,2001,162.0,920.0,920,866,0,...,2.0,5.0,0.0,1.0,4.0,2.0,1.0,4.0,4.0,2.0
3,70,9550,7,5,1915,0.0,756.0,961,756,0,...,0.0,5.0,2.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0
4,60,14260,8,5,2000,350.0,1145.0,1145,1053,0,...,2.0,5.0,0.0,1.0,4.0,2.0,1.0,4.0,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,1936,4,7,1970,0.0,546.0,546,546,0,...,5.0,5.0,2.0,1.0,4.0,3.0,2.0,4.0,4.0,2.0
2915,160,1894,4,5,1970,0.0,546.0,546,546,0,...,4.0,5.0,4.0,1.0,4.0,3.0,2.0,4.0,4.0,2.0
2916,20,20000,5,7,1960,0.0,1224.0,1224,0,0,...,0.0,5.0,0.0,1.0,4.0,3.0,2.0,4.0,4.0,2.0
2917,85,10441,5,5,1992,0.0,912.0,970,0,0,...,2.0,5.0,4.0,1.0,4.0,3.0,2.0,4.0,4.0,2.0


In [94]:
df3_train.isnull().values.any()

False

In [95]:
cormat = df3_train.corr()
sorted_mat = cormat.sort_values("SalePrice")
core = sorted_mat[['SalePrice']]
core1 = core[core['SalePrice']>= 0.2 ]
name = list(core1.index)
train_ = df3_train[name]
train_ = train_.drop(columns = 'SalePrice')
train_.head()

,Exterior2nd_VinylSd,Exterior1st_VinylSd,OpenPorchSF,WoodDeckSF,2ndFlrSF,MasVnrType_Stone,LotArea,SaleCondition_Partial,Neighborhood_NoRidge,SaleType_New,...,Foundation_PConc,YearBuilt,FullBath,TotRmsAbvGrd,1stFlrSF,TotalBsmtSF,GarageArea,GarageCars,GrLivArea,OverallQual
0,1,1,61,0,854,0,8450,0,0,0,...,1,2003,2,8,856,856.0,548.0,2.0,1710,7
1,0,0,0,298,0,0,9600,0,0,0,...,0,1976,2,6,1262,1262.0,460.0,2.0,1262,6
2,1,1,42,0,866,0,11250,0,0,0,...,1,2001,2,6,920,920.0,608.0,2.0,1786,7
3,0,0,35,0,756,0,9550,0,0,0,...,0,1915,1,7,961,756.0,642.0,3.0,1717,7
4,1,1,84,192,1053,0,14260,0,1,0,...,1,2000,2,9,1145,1145.0,836.0,3.0,2198,8


In [96]:
scaler = MinMaxScaler()
final_train = pd.DataFrame(scaler.fit_transform(train_), columns = train_.columns)
final_train.head()

,Exterior2nd_VinylSd,Exterior1st_VinylSd,OpenPorchSF,WoodDeckSF,2ndFlrSF,MasVnrType_Stone,LotArea,SaleCondition_Partial,Neighborhood_NoRidge,SaleType_New,...,Foundation_PConc,YearBuilt,FullBath,TotRmsAbvGrd,1stFlrSF,TotalBsmtSF,GarageArea,GarageCars,GrLivArea,OverallQual
0,1.0,1.0,0.082210,0.000000,0.413559,0.0,0.033420,0.0,0.0,0.0,...,1.0,0.949275,0.50,0.461538,0.109641,0.140098,0.368280,0.4,0.259231,0.666667
1,0.0,0.0,0.000000,0.209270,0.000000,0.0,0.038795,0.0,0.0,0.0,...,0.0,0.753623,0.50,0.307692,0.194917,0.206547,0.309140,0.4,0.174830,0.555556
2,1.0,1.0,0.056604,0.000000,0.419370,0.0,0.046507,0.0,0.0,0.0,...,1.0,0.934783,0.50,0.307692,0.123083,0.150573,0.408602,0.4,0.273549,0.666667
3,0.0,0.0,0.047170,0.000000,0.366102,0.0,0.038561,0.0,0.0,0.0,...,0.0,0.311594,0.25,0.384615,0.131695,0.123732,0.431452,0.6,0.260550,0.666667
4,1.0,1.0,0.113208,0.134831,0.509927,0.0,0.060576,0.0,1.0,0.0,...,1.0,0.927536,0.50,0.538462,0.170342,0.187398,0.561828,0.6,0.351168,0.777778


In [97]:
#model training
x = final_train[:1460]
x_test = final_train[1460:]
y


,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [110]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
X_train, X_val, y_train, y_val =train_test_split(x, y ,train_size=0.9, test_size=0.1,random_state=0)

model.fit(X_train,y_train)
model.score(X_train,y_train) 
predicts = model.predict(X_val)
mse = mean_squared_error(predicts,y_val) #MSE is very high
rmse = np.sqrt(mse)
print('MSE=', mse)
print('RMSE=',rmse)

LinearRegression()

0.806005521334396

MSE= 1345167814.6648297
RMSE= 36676.52947955722


In [111]:
model.score(X_train, y_train)
model.score(X_val, y_val)
#R2, from this model, it is the sigsn of underfit.

0.806005521334396

0.8042570942780851

In [112]:
LN_pred = model.predict(x_test)
LN_pred = pd.DataFrame(LN_pred, columns = ['SalePrice'])
#x_testID
LN_pred['ID'] = x_testID
LN_pred.to_csv('LN_pred.csv', index=False)

In [21]:
sfs = SFS(RandomForestClassifier(n_estimators = 100, random_state = 0, n_jobs = -1),
         k_features = 20,
         forward = True,
         verbose = 2,
         scoring = 'accuracy',
          cv = 4,
          n_jobs = -1
          ).fit(x,y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:   13.5s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   14.2s finished

[2022-04-17 20:21:51] Features: 1/20 -- score: 0.01712328767123288[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:   11.9s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   13.6s finished

[2022-04-17 20:22:04] Features: 2/20 -- score: 0.018493150684931507[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  22 | elapsed:   12.5s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   13.7s finished

[2022-04-17 20:22:18] Features: 3/20 -- score: 0.018493150684931507[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out o

In [25]:
sfs.transform(x).shape

(1460, 20)

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [27]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel_.fit(X_train, np.ravel(y_train,order='C'))
sel_.get_support()
X_train = pd.DataFrame(X_train)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

array([ True,  True, False, False,  True,  True, False,  True,  True,
        True,  True, False,  True, False,  True,  True,  True,  True,
       False, False,  True,  True, False,  True])

In [28]:
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

total features: 24
selected features: 16
features with coefficients shrank to zero: 14443


In [29]:
#removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
#removed_feats
X_train_selected = sel_.transform(X_train)
X_test_selected = sel_.transform(X_val)
X_train_selected.shape, X_test_selected.shape

((1314, 16), (146, 16))

In [43]:
from sklearn.linear_model import Ridge

ridge = Ridge().fit(X_train, y_train)


In [44]:
ridge_pred = ridge.predict(x_test)

In [45]:
ridge_pred = pd.DataFrame(ridge_pred, columns = ['SalePrice'])
#x_testID
ridge_pred['ID'] = x_testID
ridge_pred.to_csv('ridge_prediction.csv', index=False)

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [42]:
y_predz = model.predict(x_test)
y_predz1 = pd.DataFrame(y_predz, columns = ['SalePrice'])
#x_testID
y_predz1['ID'] = x_testID
y_predz1.to_csv('prediction4.csv', index=False)

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [46]:
##Lasso Regression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

Lasso_reg =Lasso()
parameters= {'alpha':[x for x in [0.0005,0.001,0.01,0.1,1]]}

Lasso_reg=GridSearchCV(Lasso_reg, param_grid=parameters)
Lasso_reg.fit(X_train,y_train)
print("The best value of Alpha is: ",Lasso_reg.best_params_,Lasso_reg.best_score_)


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 500164285487.5943, tolerance: 657118734.8147434
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 597001598469.9631, tolerance: 655974723.0327224
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 618584401174.1519, tolerance: 635955900.9288124
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarnin

GridSearchCV(estimator=Lasso(),
             param_grid={'alpha': [0.0005, 0.001, 0.01, 0.1, 1]})

The best value of Alpha is:  {'alpha': 1} 0.7740416762552802


In [47]:
Lasso_reg =Lasso(alpha=1)
Lasso_reg.fit(X_train,y_train)
y_pred_train=Lasso_reg.predict(X_train)
y_pred_test=Lasso_reg.predict(x_test)

print('Root Mean Square Error train = ' + str(np.sqrt(mean_squared_error(y_train, y_pred_train)))) 

Lasso(alpha=1)

Root Mean Square Error train = 34803.36424666584


In [48]:
y_pred_test= pd.DataFrame(y_pred_test,columns=['SalePrice'])
y_pred_test['ID'] = x_testID
y_pred_test.to_csv('prediction3.csv', index=False)

In [78]:
##Elastic Net
from sklearn.linear_model import ElasticNet 
en = ElasticNet(alpha=1.0, l1_ratio=0.5) 
#cv = KFold(n_splits=5, shuffle=True, random_state=0)
#param_grid = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10, 100], 'l1_ratio': np.arange(0, 1, 0.01), 'tol': [0.1,0.01,0.001,0.0001]}

#search = GridSearchCV(en, param_grid, scoring='r2', cv=cv, n_jobs=-1)

#results = search.fit(x, y)
en.fit(X_train, y_train) 
en_pred = en.predict(x_test)

ElasticNet()

In [79]:
en_pred= pd.DataFrame(en_pred,columns=['SalePrice'])
en_pred['ID'] = x_testID
en_pred.to_csv('prediction.csv', index=False)